In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import time



driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.redbus.in/online-booking/kaac-transport");

time.sleep(4)

scrolling = True
while scrolling:
    body = driver.find_element(By.TAG_NAME,'body')
    old_page = driver.page_source

    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(3)

    new_page = driver.page_source

    if new_page == old_page:
        scrolling = False

page1 = driver.find_elements(By.CSS_SELECTOR,'a[class="route"]')
kaac1 = pd.DataFrame(
    {
        'route' : [i.text for i in page1],
        'route_link': [i.get_attribute('href') for i in page1]
    }
)
time.sleep(4)

body = driver.find_element(By.TAG_NAME, 'body')
body.send_keys(Keys.PAGE_DOWN)
time.sleep(2) 

driver.find_element(By.XPATH, "//div[contains(text(),'2')]").click()

page2 = driver.find_elements(By.CSS_SELECTOR,'a[class="route"]')
kaac2 = pd.DataFrame(
    {
        'route' : [i.text for i in page2],
        'route_link': [i.get_attribute('href') for i in page2]
    }
)

kaac = pd.concat([kaac1,kaac2])
kaac.reset_index(inplace = True)
kaac.drop(['index'], axis = 1, inplace = True)


# Base DataFrame to collect all bus information
all_bus_data = pd.DataFrame()


for url in kaac['route_link']:
    driver.get(url);
    time.sleep(6)
    scrolling = True
    while scrolling:
        old_page = driver.page_source
        body = driver.find_element(By.TAG_NAME,"body")
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(3)
        new_page = driver.page_source
        if old_page == new_page:
            scrolling = False
        else:
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(3)
    # Extract data
    bus_route = driver.find_element(By.CSS_SELECTOR,'h1[class="D136_h1"]')
    bus_name = driver.find_elements(By.XPATH, '//div[@class="travels lh-24 f-bold d-color"]')
    bus_type = driver.find_elements(By.XPATH, '//div[@class="bus-type f-12 m-top-16 l-color evBus"]')
    departure_time = driver.find_elements(By.XPATH, '//div[@class="dp-time f-19 d-color f-bold"]')
    duration = driver.find_elements(By.XPATH, '//div[@class="dur l-color lh-24"]')
    arrival_time = driver.find_elements(By.XPATH, '//div[@class="bp-time f-19 d-color disp-Inline"]')
    star_rating = driver.find_elements(By.XPATH, '//div[@class="lh-18 rating rat-green "]')
    price = driver.find_elements(By.XPATH, '//span[@class="f-19 f-bold"]')
    seats_available = driver.find_elements(By.XPATH, '//div[@class="seat-left m-top-30"]')

    # Convert elements to lists
    bus_route_name = bus_route.text
    bus_name_lst = [i.text for i in bus_name]
    bus_type_lst = [i.text for i in bus_type]
    departure_time_lst = [i.text for i in departure_time]
    duration_lst = [i.text for i in duration]
    arrival_time_lst = [i.text for i in arrival_time]
    star_rating_lst = [i.text for i in star_rating]
    price_lst = [i.text for i in price]
    seats_available_lst = []
    for seat in seats_available:
        text = seat.text  # Get the text content
        number_of_seats = text.split()[0]  # Split and take the first part, which is the number
        seats_available_lst.append(int(number_of_seats)) 

 # Pad each list with NaN values
    max_length = max(len(bus_name_lst), len(bus_type_lst), len(departure_time_lst),
                     len(duration_lst), len(arrival_time_lst), len(star_rating_lst),
                     len(price_lst), len(seats_available_lst))

    bus_name_lst += [np.nan] * (max_length - len(bus_name_lst))
    bus_type_lst += [np.nan] * (max_length - len(bus_type_lst))
    departure_time_lst += [np.nan] * (max_length - len(departure_time_lst))
    duration_lst += [np.nan] * (max_length - len(duration_lst))
    arrival_time_lst += [np.nan] * (max_length - len(arrival_time_lst))
    star_rating_lst += [np.nan] * (max_length - len(star_rating_lst))
    price_lst += [np.nan] * (max_length - len(price_lst))
    seats_available_lst += [np.nan] * (max_length - len(seats_available_lst))

    # Create a DataFrame for the current URL
    kaac_bus = pd.DataFrame({
        'Route Name': [bus_route_name] * max_length,
        'Route Link': [url] * max_length,
        'Bus Name': bus_name_lst,
        'Bus Type': bus_type_lst,
        'Departure Time': departure_time_lst,
        'Duration': duration_lst,
        'Arrival Time': arrival_time_lst,
        'Star Rating': star_rating_lst,
        'Price': price_lst,
        'Seats Available': seats_available_lst
    })

    # Append the current DataFrame to the all_bus_data DataFrame
    all_bus_data = pd.concat([all_bus_data, kaac_bus], ignore_index=True)



# Convert 'Price' and 'Star Rating' to numeric, forcing non-numeric values to NaN
all_bus_data['Price'] = pd.to_numeric(all_bus_data['Price'], errors='coerce')
all_bus_data['Star Rating'] = pd.to_numeric(all_bus_data['Star Rating'], errors='coerce')

# Fill NaNs in these columns with their mean, rounded to the nearest integer
all_bus_data['Price'] = all_bus_data['Price'].fillna(all_bus_data['Price'].mean().round(0).astype(int))
all_bus_data['Star Rating'] = all_bus_data['Star Rating'].fillna(all_bus_data['Star Rating'].mean().round(0).astype(int))
all_bus_data['Seats Available']=all_bus_data['Seats Available'].fillna(all_bus_data['Seats Available'].mean().round(0).astype(int))


# Assign to 'a' after filling nulls
kaac_bus = all_bus_data
kaac_bus





,Route Name,Route Link,Bus Name,Bus Type,Departure Time,Duration,Arrival Time,Star Rating,Price,Seats Available
0,Dokmoka to Guwahati Bus,https://www.redbus.in/bus-tickets/dokmoka-to-g...,KAAC TRANSPORT - 157249,Non AC Seater 2+2,05:45,03h 45m,09:30,4.1,250,37.0
1,Dokmoka to Guwahati Bus,https://www.redbus.in/bus-tickets/dokmoka-to-g...,Kareng Travels,Non AC Seater (2+2),06:00,03h 30m,09:30,4.2,280,37.0
2,Dokmoka to Guwahati Bus,https://www.redbus.in/bus-tickets/dokmoka-to-g...,Kareng Travels,Bharat Benz A/C Seater (2+1),11:45,04h 15m,16:00,4.8,300,37.0
3,Dokmoka to Guwahati Bus,https://www.redbus.in/bus-tickets/dokmoka-to-g...,Kareng Travels,Non AC Seater (2+1),13:00,21h 15m,10:15,4.1,280,37.0
4,Bokolia (assam) to Guwahati Bus,https://www.redbus.in/bus-tickets/bokolia-assa...,KAAC TRANSPORT - 157249,Non AC Seater 2+2,05:00,04h 30m,09:30,4.1,299,37.0
5,Bokolia (assam) to Guwahati Bus,https://www.redbus.in/bus-tickets/bokolia-assa...,Kareng Travels,Bharat Benz A/C Seater (2+1),11:15,04h 45m,16:00,4.1,300,37.0
6,Bokolia (assam) to Guwahati Bus,https://www.redbus.in/bus-tickets/bokolia-assa...,Kareng Travels,Non AC Seater (2+1),12:30,21h 45m,10:15,4.0,300,37.0
7,Bokolia (assam) to Guwahati Bus,https://www.redbus.in/bus-tickets/bokolia-assa...,Kareng Travels,Non AC Seater (2+2),05:30,04h 00m,09:30,4.0,300,37.0
8,Guwahati to Bokolia (assam) Bus,https://www.redbus.in/bus-tickets/guwahati-to-...,KAAC TRANSPORT - 157250,Non AC Seater 2+2,11:30,04h 15m,15:45,4.2,299,37.0
9,Guwahati to Bokolia (assam) Bus,https://www.redbus.in/bus-tickets/guwahati-to-...,Kareng Travels,A/C Seater (2+1),09:30,05h 15m,14:45,4.3,300,37.0


In [ ]:

# convert to csv file
all_kaac_buses_withnul.to_csv('kaac_bus_withnull.csv', index=False)

In [4]:
kaac_bus.isnull().sum()

Route Name         0
Route Link         0
Bus Name           0
Bus Type           0
Departure Time     0
Duration           0
Arrival Time       0
Star Rating        0
Price              0
Seats Available    0
dtype: int64

In [ ]:
for url in kaac['route_link']:
    driver.get(url);
    time.sleep(6)
    scrolling = True
    while scrolling:
        old_page = driver.page_source
        body = driver.find_element(By.TAG_NAME,"body")
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(3)
        new_page = driver.page_source
        if old_page == new_page:
            scrolling = False
        else:
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(3)
    # Extract data
    bus_route = driver.find_element(By.CSS_SELECTOR,'h1[class="D136_h1"]')
    bus_name = driver.find_elements(By.XPATH, '//div[@class="travels lh-24 f-bold d-color"]')
    bus_type = driver.find_elements(By.XPATH, '//div[@class="bus-type f-12 m-top-16 l-color evBus"]')
    departure_time = driver.find_elements(By.XPATH, '//div[@class="dp-time f-19 d-color f-bold"]')
    duration = driver.find_elements(By.XPATH, '//div[@class="dur l-color lh-24"]')
    arrival_time = driver.find_elements(By.XPATH, '//div[@class="bp-time f-19 d-color disp-Inline"]')
    star_rating = driver.find_elements(By.XPATH, '//div[@class="lh-18 rating rat-green "]')
    price = driver.find_elements(By.XPATH, '//span[@class="f-19 f-bold"]')
    seats_available = driver.find_elements(By.XPATH, '//div[@class="seat-left m-top-30"]')

    # Convert elements to lists
    bus_route_name = bus_route.text
    bus_name_lst = [i.text for i in bus_name]
    bus_type_lst = [i.text for i in bus_type]
    departure_time_lst = [i.text for i in departure_time]
    duration_lst = [i.text for i in duration]
    arrival_time_lst = [i.text for i in arrival_time]
    star_rating_lst = [i.text for i in star_rating]
    price_lst = [i.text for i in price]
    seats_available_lst = []
    for seat in seats_available:
        text = seat.text  # Get the text content
        number_of_seats = text.split()[0]  # Split and take the first part, which is the number
        seats_available_lst.append(int(number_of_seats)) 


print(bus_route_name)
print(bus_name_lst)
print(bus_type_lst) 
print(departure_time_lst) 
print(duration_lst) 
print(arrival_time_lst)
print(star_rating_lst)
print(price_lst) 
print(seats_available_lst)
        
        

In [43]:
kaac['route_link']

'https://www.redbus.in/bus-tickets/diphu-to-guwahati'

'https://www.redbus.in/bus-tickets/diphu-to-guwahati'

In [52]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

kaac_route_data = []

# Loop through each URL in your dataset
for url in kaac['route_link']:
        # Reinitialize the WebDriver for each URL to avoid session disconnection issues
        driver = webdriver.Chrome()
        driver.maximize_window()
        driver.get(url)
        time.sleep(6)
        
        # Scroll through the page
        scrolling = True
        while scrolling:
            old_page = driver.page_source
            body = driver.find_element(By.TAG_NAME, "body")
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(3)
            new_page = driver.page_source
            if old_page == new_page:
                scrolling = False
            else:
                body.send_keys(Keys.PAGE_DOWN)
                time.sleep(3)
        
        # Extract data
        bus_route = driver.find_element(By.CSS_SELECTOR, 'h1[class="D136_h1"]').text
        bus_name_lst = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="travels lh-24 f-bold d-color"]')]
        bus_type_lst = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="bus-type f-12 m-top-16 l-color evBus"]')]
        departure_time_lst = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="dp-time f-19 d-color f-bold"]')]
        duration_lst = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="dur l-color lh-24"]')]
        arrival_time_lst = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="bp-time f-19 d-color disp-Inline"]')]
        star_rating_lst = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="lh-18 rating rat-green "]')]
        price_lst = [i.text for i in driver.find_elements(By.XPATH, '//span[@class="f-19 f-bold"]')]
        seats_available_lst = []
        
        for seat in driver.find_elements(By.XPATH, '//div[@class="seat-left m-top-30"]'):
            text = seat.text  # Get the text content
            number_of_seats = text.split()[0]  # Split and take the first part, which is the number
            seats_available_lst.append(int(number_of_seats))
        
       

[{'bus_route': 'Diphu to Guwahati Bus',
  'bus_name': [],
  'bus_type': [],
  'departure_time': [],
  'duration': [],
  'arrival_time': [],
  'star_rating': [],
  'price': [],
  'seats_available': []},
 {'bus_route': 'Guwahati to Diphu Bus',
  'bus_name': ['KAAC TRANSPORT - 157250', 'Kareng Travels'],
  'bus_type': ['Non AC Seater 2+2', 'Non AC Seater (2+2)'],
  'departure_time': ['11:30', '11:45'],
  'duration': ['06h 15m', '06h 45m'],
  'arrival_time': ['17:45', '18:30'],
  'star_rating': ['4.5', '4.5'],
  'price': [],
  'seats_available': []},
 {'bus_route': 'Dokmoka to Guwahati Bus',
  'bus_name': [],
  'bus_type': [],
  'departure_time': [],
  'duration': [],
  'arrival_time': [],
  'star_rating': [],
  'price': [],
  'seats_available': []},
 {'bus_route': 'Bokolia (assam) to Guwahati Bus',
  'bus_name': [],
  'bus_type': [],
  'departure_time': [],
  'duration': [],
  'arrival_time': [],
  'star_rating': [],
  'price': [],
  'seats_available': []},
 {'bus_route': 'Guwahati to Bok

In [56]:
max_length = max(len(bus_name_lst), len(bus_type_lst), len(departure_time_lst),
                     len(duration_lst), len(arrival_time_lst), len(star_rating_lst),
                     len(price_lst), len(seats_available_lst))

bus_name += [np.nan] * (max_length - len(bus_name_lst))
bus_type_lst += [np.nan] * (max_length - len(bus_type_lst))
departure_time_lst += [np.nan] * (max_length - len(departure_time_lst))
duration_lst += [np.nan] * (max_length - len(duration_lst))
arrival_time_lst += [np.nan] * (max_length - len(arrival_time_lst))
star_rating_lst += [np.nan] * (max_length - len(star_rating_lst))
price_lst += [np.nan] * (max_length - len(price_lst))
seats_available_lst += [np.nan] * (max_length - len(seats_available_lst))



kaac__ = pd.DataFrame(
    {
        "bus_route": bus_route,
        "bus_name": bus_name_lst,
        "bus_type": bus_type_lst,
        "departure_time": departure_time_lst,
        "duration": duration_lst,
        "arrival_time": arrival_time_lst,
        "star_rating": star_rating_lst,
        "price": price_lst,
        "seats_available": seats_available_lst
            
    }
)

NameError: name 'bus_name' is not defined

In [23]:
print(bus_route_name)
print(bus_name_lst)
print(bus_type_lst) 
print(departure_time_lst) 
print(duration_lst) 
print(arrival_time_lst)
print(star_rating_lst)
print(price_lst) 
print(seats_available_lst)

Guwahati to Hamren Bus
[]
[]
[]
[]
[]
[]
[]
[]
